Importing the dependencies


In [ ]:
import pandas as pd
import numpy as np 
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

importing the data


In [31]:
data = pd.read_csv(f'datasets/train.csv')
test_data = pd.read_csv(f'datasets/test.csv')

In [32]:
data.head()

,id,plate,date,price
0,1,X059CP797,2024-12-26 00:00:00,65000
1,2,Y800MH790,2024-07-12 21:31:37,100000
2,3,A212TX77,2024-04-18 00:00:00,290000
3,4,P001AY199,2025-01-03 00:27:15,680000
4,5,P001AY199,2025-01-10 09:32:41,750000


In [33]:
data.isnull().sum()

id       0
plate    0
date     0
price    0
dtype: int64

In [34]:
data

,id,plate,date,price
0,1,X059CP797,2024-12-26 00:00:00,65000
1,2,Y800MH790,2024-07-12 21:31:37,100000
2,3,A212TX77,2024-04-18 00:00:00,290000
3,4,P001AY199,2025-01-03 00:27:15,680000
4,5,P001AY199,2025-01-10 09:32:41,750000
...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000
51631,51632,M004KA161,2025-01-31 00:00:00,1600000
51632,51633,E888EB199,2025-02-06 00:00:00,850000
51633,51634,X023XK77,2024-04-24 11:30:07,150000


In [35]:
test_data.isnull().sum()

id          0
plate       0
date        0
price    7695
dtype: int64

In [36]:
data.drop(columns=['id'],inplace=True)
test_data.drop(columns=['id','price'],inplace=True)


In [37]:
data

,plate,date,price
0,X059CP797,2024-12-26 00:00:00,65000
1,Y800MH790,2024-07-12 21:31:37,100000
2,A212TX77,2024-04-18 00:00:00,290000
3,P001AY199,2025-01-03 00:27:15,680000
4,P001AY199,2025-01-10 09:32:41,750000
...,...,...,...
51630,X023PP797,2025-01-29 00:00:00,70000
51631,M004KA161,2025-01-31 00:00:00,1600000
51632,E888EB199,2025-02-06 00:00:00,850000
51633,X023XK77,2024-04-24 11:30:07,150000


In [38]:
data.nunique()

plate    43605
date      9092
price      798
dtype: int64

In [39]:
test_data.isnull().sum()

plate    0
date     0
dtype: int64

In [40]:
test_data

,plate,date
0,P700TT790,2025-01-27 00:00:00
1,M081TX797,2025-02-10 00:00:00
2,T333HX777,2025-02-11 00:00:00
3,H744BH977,2025-02-03 00:00:00
4,X066EM777,2025-02-12 00:00:00
...,...,...
7690,X799CC799,2023-03-13 00:00:00
7691,K077YT777,2022-10-19 00:00:00
7692,O200PA777,2023-10-02 19:19:30
7693,O073OO97,2025-01-12 00:00:00


Lets us split the data in x and y

In [41]:
X = data.drop(columns='price')
Y = data['price']

In [42]:
X_train , X_test , Y_train , Y_test = train_test_split(X ,Y , random_state=2, test_size=0.2)

Model Implementation

In [43]:
model = CatBoostRegressor(iterations=2,
                          depth=2,
                          learning_rate=1,
                          loss_function='RMSE')

In [44]:
categorical_features = ['plate','date']

In [45]:
model.fit(X_train,Y_train, cat_features=categorical_features)

0:	learn: 1854266.4088691	total: 16.9ms	remaining: 16.9ms
1:	learn: 1852709.4850413	total: 23.2ms	remaining: 0us


In [47]:
print(model.predict(X_test))

[290652.74048345 755799.90390897 499293.34347485 ... 290652.74048345
 425745.38628057 290652.74048345]


On test_data

In [53]:
prediction_data = model.predict(test_data)

Preparing the submission csv

In [52]:
sample_data = pd.read_csv(f'datasets/test.csv')

In [55]:
Final_prediction = pd.DataFrame({
    'id': sample_data['id'][:len(prediction_data)],  # Align lengths
    'price': prediction_data
})


In [56]:
Final_prediction

,id,price
0,51636,499293.343475
1,51637,499293.343475
2,51638,499293.343475
3,51639,290652.740483
4,51640,499293.343475
...,...,...
7690,59326,499293.343475
7691,59327,499293.343475
7692,59328,499293.343475
7693,59329,290652.740483


In [57]:
Final_prediction.to_csv(f'datasets/Final_Submission.csv',index=False)